## Preprocessing the Raw Data Underpinning `tell`

In this notebook we will load and process the necessary data for `tell`. The raw data (i.e., historical loads, population, meteorology, etc.) is in a variety of different formats determined by their originating organization (e.g., the Energy Information Agency). This series of processing steps cleans the raw data and converts it into a set of simplified .csv files which can be combined together and used as input to the MLP model training steps in `tell`. Follow the sequence below to pre-process the `tell` input data.

### Install the package of data underpinning `tell`

`tell` is based on open-source publicly accessible data. For convienence, we've packaged all of the core data underpinning tell into a [Zenodo data package](https://zenodo.org/record/5714756#.YhkTjxPMJTY). In order to run this notebook, first set the local directory where you would like to store the package data and the run the `install_tell_raw_data` function below. Note that the raw data package will require ~1.6 GB of storage and can take several minutes to download.

In [ ]:
# Start by importing the `tell` package and information about your operating system:
import os 
import tell


In [ ]:
# Identify the current working directory, the subdirectory where the data will be stored, and the image output subdirectory:
current_dir =  os.path.join(os.path.dirname(os.getcwd()))
tell_data_dir = os.path.join(current_dir, r'tell_data')
tell_image_dir = os.path.join(tell_data_dir, r'visualizations')

# If the "tell_data_dir" subdirectory doesn't exist then create it:
if not os.path.exists(tell_data_dir):
   os.makedirs(tell_data_dir)

# If the "tell_image_dir" subdirectory doesn't exist then create it:
if not os.path.exists(tell_image_dir):
   os.makedirs(tell_image_dir)


In [ ]:
# Download the TELL raw data package from Zenodo:
tell.install_tell_raw_data(data_dir = tell_data_dir)


### Spatially mapping BAs to counties

The code block underpins the spatial component of `tell`. The function `map_ba_service_territory` uses information about Balancing Authorities (BAs) from the [EIA-861](https://www.eia.gov/electricity/data/eia861/) dataset to spatially map BAs to U.S. counties. This allows us to assign loads projected for individual BAs in `tell` to where they would occur with respect to each other spatially. More information about how BAs are mapped is available on the [Read the Docs](https://immm-sfa.github.io/tell/index.html) site for `tell`.

In [ ]:
# Execute the mapping function for all years from 2015 to 2020:
tell.map_ba_service_territory(start_year = 2015,
                              end_year = 2020,
                              data_input_dir = tell_data_dir)


In [ ]:
# See the output of the mapping step by plotting the service territory for a given BA (e.g., PJM, CISO, ERCO, etc.) in a given year (e.g., 2015-2020):
tell.plot_ba_service_territory(ba_to_plot = 'PACE',
                               year_to_plot = '2019',
                               data_input_dir = tell_data_dir, 
                               image_output_dir = tell_image_dir,
                               image_resolution = 150,
                               save_images = False)


### Process the EIA-930 historical hourly demand dataset

Here we convert the raw EIA-930 hourly electricity demand data for BAs from Excel files to .csv files. The function `process_eia_930_data` also subsets the EIA-930 dataset to only include the desired variables. Note that this step can take a few minutes even when using parallel processing streams.

In [ ]:
# Execute the function to pre-proceess the raw EIA-930 data using parallel processing streams:
tell.process_eia_930_data(data_input_dir = tell_data_dir,
                          n_jobs = -1)


### Process the historical population dataset

This pre-processing step takes historical county-level population data from the U.S. Census Bureau and computes the total population living within the service territory of each BA. The historical annual populations are then interpolated to an hourly resolution in preparation to use population as input to the MLP models.

In [ ]:
# Execute the function to pre-process the historical observed population data for all years from 2015 to 2020:
tell.process_ba_population_data(start_year = 2015,
                                end_year = 2020,
                                data_input_dir = tell_data_dir)


### Download the sample historical and future meteorology data

This pre-processing step relies on another pre-packaged sample dataset containing historical and sample future weather data used in `tell`. This sample forcing data is also available from a [Zenodo data package](https://zenodo.org/record/6354665#.Yi-_PRDMJTY). This section downloads that sample forcing data. The historical and future meteorology is based on IM3 simulations using the Weather Research and Forecasting (WRF) model. Meteorological output from WRF is first averaged into county-level mean values and then population-weighted to create an hourly time series of meteorology for each BA in `tell`.  Note that the sample weather data package will require ~250 MB of storage and can take several minutes to download.

In [ ]:
# Download the TELL sample forcing data package from Zenodo:
tell.install_sample_forcing_data(data_dir = tell_data_dir)


### Compile the historical hourly load, population, and meteorology data 

In this final pre-processing step we compile the historical load, population, and meteorology data into a single set of .csv files. There is one .csv file generated for each BA in `tell`. These composite .csv files are then used as input to the MLP model training. Note that this step can take a few minutes.

In [ ]:
# Compile the historical load, population, and meteorology data into a single set of .csv files:
tell.compile_data(start_year = 2015,
                  end_year = 2019,
                  data_input_dir = tell_data_dir)
